![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-large.png)  
[National Science Data Fabric](https://nationalsciencedatafabric.org/) 

# Converting Nexus data 


In [ ]:
import os ,sys, time, logging,shutil,copy
from datetime import datetime
import numpy as np

# import nexus
# python3 -m pip install nexusformat
# https://github.com/nexpy/nexusformat/blob/main/src/nexusformat/nexus/tree.py

from nexusformat.nexus import * 
from nexusformat.nexus.tree import NX_CONFIG 
NX_CONFIG['memory']=16000 # alllow data to be 16000MB (i.e. 16GB)

# import openvisus
if os.path.isdir(r"C:\projects\OpenVisus\build\RelWithDebInfo"):
	sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

import OpenVisus as ov
os.environ["VISUS_DISABLE_WRITE_LOCK"]="1"
logger= logging.getLogger("OpenVisus")
if False: ov.SetupLogger(logger, stream=True) # for debugging

# ////////////////////////////////////////////////////////////
class ConvertNexus:

	# constructor
	def __init__(self,src, dst, compression="raw",streamable=None):
		self.src=src
		self.dst=dst
		self.streamable=streamable
		self.compression=compression
		self.num_bin_fields=0

	@staticmethod
	def traverse(cur,nrec=0):
		yield (nrec,cur)

		try:
			childs=cur.entries.items()
		except:
			return

		for _k, child in childs: 
			yield from ConvertNexus.traverse(child,nrec+1)

	@staticmethod
	def print(cur):
		for depth, node in ConvertNexus.traverse(cur):
			if isinstance(node,NXfield) and not isinstance(node,NXlinkfield) and(len(node.shape)==3 or len(node.shape)==4):
				print("   "*(depth+0) +  f"{node.nxname}::{type(node)} shape={node.shape} dtype={node.dtype} ***********************************")
			else:
				print("   "*(depth+0) + f"{node.nxname}::{type(node)}")
			for k,v in node.attrs.items():
				print("  "*(depth+1) + f"@{k} = {v}")

	# run
	def run(self):
		print(f"ConvertNexus::run src={self.src} full-size={os.path.getsize(self.src):,}")
		src=nxload(self.src)
		dst=copy.deepcopy(src)
		dst.attrs["streamable"]=True
		self._convertNexusFieldsToOpenVisus(src, dst)

		if self.streamable:
			t1=time.time()
			print(f"Creating streamable version {self.streamable}")
			if os.path.isfile(self.streamable): os.remove(self.streamable)			
			nxsave(self.streamable, dst , mode='w')
			print(f"ConvertNexus::run streamable={self.streamable} reduced-size={os.path.getsize(self.streamable):,}")
		return dst
	
	# _convertNexusFieldsToOpenVisus
	def _convertNexusFieldsToOpenVisus(self, src, dst):

		if isinstance(src,NXfield) and not isinstance(src,NXlinkfield):

			src_field=src;src_parent=src_field.nxgroup
			dst_field=dst;dst_parent=dst_field.nxgroup

			if len(src_field.shape)==3 or len(src_field.shape)==4:

				# TODO: support more than once
				assert(self.num_bin_fields==0)

				# replace any 'big' field with something virtually empty
				# TODO: read nexus by slabs
				t1=time.time()
				print(f"Reading Nexus field name={src_field.nxname} dtype={src_field.dtype} shape={src_field.shape} ...")
				data = src_field.nxdata
				vmin,vmax=np.min(data),np.max(data)
				print(f"Read Nexus field in {time.time()-t1} seconds vmin={vmin} vmax={vmax}")

				# e.g. 1x1441x676x2048 -> 1x1441x676x2048
				if len(data.shape)==4:
					assert(data.shape[0]==1)
					data=data[0,:,:,:]

				t1=time.time()
				print(f"Creating IDX file {self.dst}")
				ov_field=ov.Field.fromString(f"""DATA {str(src_field.dtype)} format(row_major) min({vmin}) max({vmax})""")

				assert(isinstance(src_parent,NXdata))

				if "axes" in src_parent.attrs:
					idx_axis=[]
					idx_physic_box=[]
					axis=[src_parent[it] for it in src_parent.attrs["axes"]]
					for it in axis:
						idx_physic_box=[str(it.nxdata[0]),str(it.nxdata[-1])] + idx_physic_box
						idx_axis=[it.nxname] +idx_axis
					idx_axis=" ".join(idx_axis)
					print("Found axis={idx_axis} idx_physic_box={idx_physic_box}")
					idx_physic_box=" ".join(idx_physic_box)
				else:
					idx_axis="X Y Z"
					D,H,W=data.shape
					idx_physic_box=f"0 {W} 0 {H} 0 {D}"

				db=ov.CreateIdx(
					url=self.dst, 
					dims=list(reversed(data.shape)), 
					fields=[ov_field], 
					compression="raw",
					physic_box=ov.BoxNd.fromString(idx_physic_box),
					axis=idx_axis
				)

				t1=time.time()
				print(f"Writing IDX data...")
				db.write(data)
				print(f"Wrote IDX data in {time.time()-t1} seconds")

				if self.compression and self.compression!="raw":
					t1 = time.time()
					print(f"Compressing dataset compression={self.compression}...")
					db.compressDataset([self.compression])
					print(f"Compressed dataset to {self.compression} in {time.time()-t1} seconds")

				# this is the version without any data
				dst_field=NXfield(value=None, shape=src_field.shape, dtype=src_field.dtype)
				dst_field.attrs["openvisus"]=repr([self.dst])
				dst_parent[src_field.nxname]=dst_field

			else:

				# deepcopy does not seem to copy nxdata (maybe for the lazy evalation?)
				dst_field.nxdata=copy.copy(src_field.nxdata)
		
		# recurse
		try:
			childs=src.entries
		except:
			childs=[]
		for name in childs:
				src_child=src.entries[name]
				dst_child=dst.entries[name]
				self._convertNexusFieldsToOpenVisus(src_child, dst_child)

print(time.time(),"all Nexus utils defined")

In [ ]:
nx=nxload("/nfs/chess/scratch/user/rv43/2023-2/id3a/shanks-3731-a/ti-2-exsitu/reduced/reduced_data.nxs")
print(nx.tree)
ConvertNexus.print(nx)

# Create Streamable 

In [ ]:
t1=time.time()

if False:
	streamable=ConvertNexus(
		"/mnt/data1/nsdf/3scans_HKLI.nxs", 
		"/mnt/data1/nsdf/remove-me/3scans_HKLI/visus.idx",
		streamable="/mnt/data1/nsdf/remove-me/3scans_HKLI/3scans_HKLI.streamable.nxs", 
		compression="raw"
	).run()

if False:
	streamable=ConvertNexus(
		"/nfs/chess/scratch/user/rv43/2023-2/id3a/shanks-3731-a/ti-2-exsitu/reduced/reduced_data.nxs", 
		"/mnt/data1/nsdf/remove-me/reduced/visus.idx",
		streamable=None, # problem here, streamable is too big (probably a problem with NXlinkfield) 
		compression="zip"
	).run()

if True:
	streamable=ConvertNexus(
		"/nfs/chess/scratch/user/rv43/2023-2/id3a/shanks-3731-a/ti-2-exsitu/reduced/reconstructed_data.nxs", 
		"/mnt/data1/nsdf/remove-me/reconstructed/visus.idx",
		streamable=None, # problem here, streamable is too big (probably a problem with NXlinkfield) 
		compression="zip"
	).run()

print(f"Created streamable in {time.time()-t1} seconds")

# NEXUS plot

Notes:
- using the **streamable version** as much as I can to show they are equivalent

In [ ]:
# find an item with axes and signal as childs
# note: nxdata is the entry in the nexus tree containing axes and signal
nxdata=[node for depth, node in CreateStreamableNexus.traverse(streamable) if isinstance(node,NXdata) and "axes" in node.attrs and "signal" in node.attrs][0]
 
axis=[nxdata[it] for it in nxdata.attrs["axes"]]
assert(all([isinstance(it,NXfield) for it in axis]))
H,K,L=axis

signal=nxdata[nxdata.attrs["signal"]]
assert(isinstance(signal,NXfield))

print(f"Nexus load done in {time.time()-t1} seconds dtype={signal.dtype} shape={signal.shape}")

project=[
	(0,1,2), 
	(1,0,2), 
	(2,0,1)
]

# ranges are in 'physical coordinates'
ranges=[(axis[I].nxdata[0], axis[I].nxdata[-1]) for I in range(3)]
print(ranges)

D,H,W=signal.shape
Y1,Y2,MY= 0, D, D//2
Z1,Z2,MZ= 0, H, H//2
X1,X2,MX= 0, W, W//2

# Example of plot with proper names/limits

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# ///////////////////////////////////////////////////////////////////
def ShowSlice(A, img,figsize=(12, 12), cmap="viridis", log=True):

	axis_name=axis[A].nxname
	Z,Y,X=project[A]

	fig, ax = plt.subplots(figsize=figsize)
	ax.set_title(axis_name)
	
	y1,y2=ranges[Y];ax.set_ylabel(axis[Y].nxname)
	x1,x2=ranges[X];ax.set_xlabel(axis[X].nxname)
	
	# todo other cases
	assert(log) 
	vmin = np.nanmin(img[img > -np.inf])
	vmax = np.nanmax(img[img <  np.inf])
	
	norm=colors.LogNorm(max(vmin, 0.01), max(vmax, 0.01))
	pos=ax.imshow(np.flip(img,axis=0),origin="upper", norm=norm, cmap=cmap,  extent=[x1,x2,y1,y2])

	ax.set_aspect('equal')
	ax.set_xlim(x1,x2)
	ax.set_ylim(y1,y2)
	fig.colorbar(pos, ax=ax,location='right')
	plt.autoscale(True)
	plt.show()

# this is the wy we query the original data

t1 = time.time()
original=nxload(r'C:\visus_datasets\3scans_HKLI.nxs')
original_data = original.entry.data.counts
print(f"Loaded original in {time.time()-t1} seconds")
print(original.tree)

ShowSlice(0,original_data[MZ,:,:]) 
ShowSlice(1,original_data[:,MY,:]) 
ShowSlice(2,original_data[:,:,MX]) 

In [ ]:
# retrieving OpenVisus dataset from the signal
idx_filename=eval(signal.attrs["openvisus"])[0]
db=ov.LoadDataset(idx_filename)

# this is the way we query the streamable data
ShowSlice(0,db.read(x=[X1,X2  ], y=[Y1,Y2  ],z=[MZ,MZ+1], num_refinements=1)[0,:,:])
ShowSlice(1,db.read(x=[X1,X2  ], y=[MY,MY+1],z=[Z1,Z2  ], num_refinements=1)[:,0,:])
ShowSlice(2,db.read(x=[MX,MX+1], y=[Y1,Y2  ],z=[Z1,Z2  ], num_refinements=1)[:,:,0])

# Show coarse to fine

In [ ]:
import os,sys
for vol in db.read(x=[X1,X2],y=[Y1,Y2],z=[MZ,MZ+1], num_refinements=3):
	print("data ready",vol.dtype,vol.shape)
	ShowSlice(0 ,vol[0,:,:])

# Get from OpenVisus server

You need to 

- add the dataset to the `/nfs/chess/nsdf01/openvisus/lib64/python3.6/site-packages/OpenVisus/visus.config`
- launch the Open Visus server


if you want to enable dynmamic dataset  you can do for example:

```
<visus> 

<Configuration> 
    <ModVisus> 
      <Dynamic enabled="true"  filename="/nfs/chess/nsdf01/openvisus/lib64/python3.6/site-packages/OpenVisus/visus.config" msec="10000" /> 
    </ModVisus>
</Configuration>

<datasets>
    <dataset 
       name='recon_combined_1_fullres-arco-1mb' 
       url='/mnt/data1/nsdf/visus-datasets/chess/recon_combined_1_fullres/arco/1mb/zip/visus.idx'  />
</datasets>

</visus>
```

In [ ]:
def GetExportsFromScript(filename):
	ret={}
	with open(filename,"rt") as file:
		for v in [line.split() for line in file.readlines()]:
			if len(v)==2 and v[0].lower()=="export":
				v=v[1].split("=",1)
				if len(v)==2: ret[v[0].strip()]=v[1].strip()
	return ret

# Get credentials to access the OpenVisus server
exports=GetExportsFromScript("../../mod_visus.identity.sh")
db=ov.LoadDataset(f"https://nsdf01.classe.cornell.edu/mod_visus?dataset=3scans_HKLI&~auth_username={exports['MODVISUS_USERNAME']}&~auth_password={exports['MODVISUS_PASSWORD']}")
vol=db.read(x=[X1,X2],y=[Y1,Y2],z=[MZ,MZ+1], num_refinements=1)
slice=vol[0,:,:]
print("data ready")
ShowSlice(0,slice)